In [1]:
import matplotlib.pyplot as plt
import numpy as np
import open3d as o3d
import trimesh as tm
import laspy
import os
from pathlib import Path

pfad = "/home/jhm/Desktop/Arbeit/ConvexNeuralVolume"


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [3]:
# Directly read and write las
las = laspy.read('/home/jhm/Downloads/Befahrung2018_reduced/out_final_resample_first_return_only.las')
print(las.header)
print(las.header.point_count)
point_format = las.point_format
print(list(point_format.dimension_names))
#las = pylas.convert(las, point_format_id=2)

<LasHeader(1.2, <PointFormat(3, 0 bytes of extra dims)>)>
43138313
['X', 'Y', 'Z', 'intensity', 'return_number', 'number_of_returns', 'scan_direction_flag', 'edge_of_flight_line', 'classification', 'synthetic', 'key_point', 'withheld', 'scan_angle_rank', 'user_data', 'point_source_id', 'gps_time', 'red', 'green', 'blue']


In [2]:
def loadPoints(x_block,y_block,z_block):
    my_file = Path(pfad+"/blocks/{}x{}y{}z.npy".format(x_block,y_block,z_block))
    if my_file.is_file():
        points = np.load(my_file)
        return points

# get min and max of all blocks

In [59]:
import os
minBound = np.ones(3) * 9999999
maxBound = np.ones(3) * 0
for name in os.listdir(pfad+"/blocks"):
    x = int(name.split("x")[0])
    y = int(name.split("x")[1].split("y")[0])
    z = int(name.split("x")[1].split("y")[1].split("z")[0])
    if x < minBound[0]:
        minBound[0] = x
    if y < minBound[1]:
        minBound[1] = y
    if z < minBound[2]:
        minBound[2] = z
    if x > maxBound[0]:
        maxBound[0] = x
    if y > maxBound[1]:
        maxBound[1] = y
    if z > maxBound[2]:
        maxBound[2] = z

In [60]:
minBound

array([ 0., -2., 12.])

In [61]:
maxBound

array([1170., 1457.,  237.])

# constrain pointcloud to a small region to work with

In [3]:
points = loadPoints(1048,1136,84)
for x in range(-15+1048,15+1048):
    for y in range(-15+1136,15+1136):
        for z in range(-15+84,15+84):
            points_ = loadPoints(x,y,z)
            if points_ is not None:
                points = np.concatenate([points,points_],0)


len(points)

33146

In [4]:
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(points[:,:3])
pcd.colors = o3d.utility.Vector3dVector(points[:,3:])
o3d.visualization.draw_geometries([pcd])

# create voxels and grow them to create rudimentary "inner" geometry
Then create starting points for neural Volumes  
Then create empty Points for the Pointcloud  
Then divide the Pointcloud into Pointblocks 

In [5]:
print("Statistical oulier removal")
cl, ind = pcd.remove_statistical_outlier(nb_neighbors=20,
                                                    std_ratio=2.0)
print("Radius oulier removal")
cl, ind = cl.remove_radius_outlier(nb_points=2, radius=20.05)

Statistical oulier removal
Radius oulier removal


In [6]:
o3d.visualization.draw_geometries([cl])

In [7]:
def createVoxelPointsXYDict(voxels):
    idx1 = np.argsort(voxels[:,2])
    idx2 = np.argsort(voxels[idx1][:,1],kind='mergesort')
    idx3 = np.argsort(voxels[idx1][idx2][:,0],kind='mergesort')
    voxels=voxels[idx1][idx2][idx3]
    voxelIdx, firstIndex, counts = np.unique(voxels[:,:2],axis=0, return_index=True, return_counts=True)
    voxelmap = {}
    index = 0
    for x in range(int(voxelIdx.max(axis=0)[0]+1)):
        row = {}
        voxelmap[x] = row
        for y in range(int(voxelIdx.max(axis=0)[1]+1)):
            column = {"min":0,"max":0,"voxels":[]}
            row[y] = column
            if index == len(voxelIdx):
                continue
            if voxelIdx[index][0] == x and voxelIdx[index][1] == y:
                minZ = 999999
                maxZ = 0
                for voIdx in range(counts[index]):
                    column["voxels"].append(voxels[firstIndex[index] + voIdx])
                    if column["voxels"][-1][2] < minZ:
                        minZ = column["voxels"][-1][2]
                    if column["voxels"][-1][2] > maxZ:
                        maxZ = column["voxels"][-1][2]
                column["max"] = maxZ
                column["min"] = minZ
                index += 1
                
    return voxelmap, voxelIdx

def fillVoxelColumns(voxelmap, voxelIdx):
    twoDMap = np.zeros(voxelIdx.max(axis=0)+1)
    twoDMap[np.swapaxes(voxelIdx,1,0)[0],np.swapaxes(voxelIdx,1,0)[1]] += 1
    fillstart = np.stack((1-twoDMap).nonzero(),-1)
    #create fillColumns
    newVoxels = []
    x_max, y_max = voxelIdx.max(axis=0)
    notfilledyet = []
    tempFillmap = {}
    for i in range(len(fillstart)):
        x,y = fillstart[i]
        #check n8 neighbourhood
        z_values = []
        for x_check in [max(0,x-1),x,min(x_max,x+1)]:
            for y_check in [max(0,y-1),y,min(y_max,y+1)]:
                if len(voxelmap[x_check][y_check]["voxels"]) > 0:
                    for voxel in voxelmap[x_check][y_check]["voxels"]:
                        if voxel[2] not in z_values:
                            z_values.append(voxel[2])
        for z_value in z_values:
            newVoxels.append([x,y,z_value])
            if (x,y) in tempFillmap:
                tempFillmap[(x,y)].append(z_value)
            else:
                tempFillmap[(x,y)] = [z_value]
        if len(z_values) == 0:
            notfilledyet.append([x,y])
    #fill columns again:
    for z in range(3):
        for coord in notfilledyet:
            if (coord[0],coord[1]) not in tempFillmap:
                z_vals = []
                for x_check in [max(0,coord[0]-1),coord[0],min(x_max,coord[0]+1)]:
                    for y_check in [max(0,coord[1]-1),coord[1],min(y_max,coord[1]+1)]:
                         if (x_check,y_check) in tempFillmap:
                                for z_value in tempFillmap[(x_check,y_check)]:
                                    if z_value not in z_vals:
                                        z_vals.append(z_value)
                                        newVoxels.append([coord[0],coord[1],z_value])
                tempFillmap[(coord[0],coord[1])] = z_vals
    newVoxels = np.array(newVoxels)
    return newVoxels

def fillVoxelMinimalDown(voxelmap, voxelIdx):
    x_max, y_max = voxelIdx.max(axis=0)
    newVoxels = []
    for x in voxelmap.keys():
        for y in voxelmap[0].keys():
            if len(voxelmap[x][y]["voxels"]) > 0:
                #we have voxels (so it will get filled by "fillColumns")
                z_start = voxelmap[x][y]["min"]
                #now do like the inverse column fill
                #check neighbourhood
                z_min = z_start
                rangeNr = 11
                for x_check in range(rangeNr):#[max(0,x-1),x,min(x_max,x+1)]:
                    for y_check in range(rangeNr):#[max(0,y-1),y,min(y_max,y+1)]:
                        x_check = min(max(0,x-x_check+rangeNr//2),x_max)
                        y_check = min(max(0,y-y_check+rangeNr//2),y_max)
                        if voxelmap[x_check][y_check]["min"] < z_min:
                            z_min = voxelmap[x_check][y_check]["min"]
                for i in range(int(z_start-z_min)):
                      newVoxels.append([x,y,z_start-1-i]) 
    newVoxels = np.array(newVoxels)
    return newVoxels
                   

def translateOldVoxels2New(oldNewVoxels):
    newVoxels = np.zeros((oldNewVoxels.shape[0]*8,oldNewVoxels.shape[1]))
    for i in range(len(oldNewVoxels)):
        x_old,y_old,z_old = oldNewVoxels[i]
        newVoxels[i*8] = np.array((x_old*2,y_old*2,z_old*2))
        newVoxels[i*8+1] = np.array((x_old*2,y_old*2,z_old*2+1))
        newVoxels[i*8+2] = np.array((x_old*2,y_old*2+1,z_old*2))
        newVoxels[i*8+3] = np.array((x_old*2,y_old*2+1,z_old*2+1))
        newVoxels[i*8+4] = np.array((x_old*2+1,y_old*2+1,z_old*2))
        newVoxels[i*8+5] = np.array((x_old*2+1,y_old*2+1,z_old*2+1))
        newVoxels[i*8+6] = np.array((x_old*2+1,y_old*2,z_old*2))
        newVoxels[i*8+7] = np.array((x_old*2+1,y_old*2,z_old*2+1))
    return newVoxels

def fillVoxelsFurther(newVoxels, oldvoxels):
    '''Fills a neighbour Voxel if there is a filled neighbour in the old scale "newFillMap" and a neighbour in the new map'''
    secondVoxels = []
    x_max, y_max = voxelIdx.max(axis=0)
    for i in range(len(newVoxels)):
        x,y,z = newVoxels[i,:]
        for x_check in [max(0,x-1),x,min(x_max,x+1)]:
            for y_check in [max(0,y-1),y,min(y_max,y+1)]:
                if len(voxelmap[x_check][y_check]["voxels"]) == 0: #not self occupied
                    hasNeighbourInNewMap = False
                    hasNeighbourInOldMap = False
                    nrOfNeighbours = 0
                    for x_check2 in [max(0,x_check-1),x,min(x_max,x_check+1)]:
                        for y_check2 in [max(0,y_check-1),y,min(y_max,y_check+1)]:
                            if len(voxelmap[x_check2][y_check2]["voxels"]) > 0: #has a real neighbour
                                for vx in voxelmap[x_check2][y_check2]["voxels"]:
                                    if vx[2] == z:
                                        secondVoxels.append([x_check,y_check,z])
                                        nrOfNeighbours += 1
    return secondVoxels

def fillVoxelsFurther2(newVoxels):
    '''Fills a neighbour Voxel if there are at least 4 neighbours present'''
    secondVoxels = []
    voxelmap,voxelIdx = createVoxelPointsXYDict(newVoxels)
    x_max, y_max = voxelIdx.max(axis=0)
    for i in range(len(newVoxels)):
        x,y,z = newVoxels[i,:]
        for x_check in [max(0,x-1),x,min(x_max,x+1)]:
            for y_check in [max(0,y-1),y,min(y_max,y+1)]:
                if len(voxelmap[x_check][y_check]["voxels"]) == 0: #not self occupied
                    #start checking n8 for 4 or more neighbours
                    nrOfNeighbours = 0
                    for x_check2 in [max(0,x_check-1),x,min(x_max,x_check+1)]:
                        for y_check2 in [max(0,y_check-1),y,min(y_max,y_check+1)]:
                            if len(voxelmap[x_check2][y_check2]["voxels"]) > 0: #has a real neighbour
                                for vx in voxelmap[x_check2][y_check2]["voxels"]:
                                    if vx[2] == z:
                                        nrOfNeighbours += 1
                    if nrOfNeighbours > 2:
                        secondVoxels.append([x_check,y_check,z])
    return secondVoxels

def voxelGrid2Voxels(voxelgrid):
    voxels = []
    for i in voxelgrid.get_voxels():
        voxels.append(i.grid_index)
    voxels = np.stack(voxels)
    return voxels

def voxelGrid2Colors(voxelgrid):
    voxels = []
    for i in voxelgrid.get_voxels():
        voxels.append(i.color)
    voxels = np.stack(voxels)
    return voxels

In [8]:
voxelSize = 32.0

voxel_grid = o3d.geometry.VoxelGrid.create_from_point_cloud(cl,
                                                            voxel_size=voxelSize)


o3d.visualization.draw_geometries([voxel_grid])



In [9]:
oldVoxels = []

rangemax = 4
for i in range(rangemax):
    print("doing run",i)
    voxel_grid = o3d.geometry.VoxelGrid.create_from_point_cloud(cl,
                                                                voxel_size=voxelSize)
    voxels = voxelGrid2Voxels(voxel_grid)
    print(voxels.max(axis=0))
    voxelmap,voxelIdx = createVoxelPointsXYDict(voxels)
    print("filling")
    newVoxels = fillVoxelColumns(voxelmap,voxelIdx)
    if len(oldVoxels) > 0:
        newVoxels = np.concatenate((newVoxels,oldVoxels))
    print("filling further")
    #secondVoxels = fillVoxelsFurther2(newVoxels)
    #newVoxels = np.concatenate((newVoxels,secondVoxels))
    newVoxels = np.unique(newVoxels,axis=0)
    if i < rangemax-1:
        print("translating")
        print(newVoxels.shape)
        if len(newVoxels) > 0:
            newVoxels = translateOldVoxels2New(newVoxels)
    oldVoxels = newVoxels
    print("done")
    voxelSize = voxelSize * 0.5
    
voxelSize = voxelSize*2
voxelmap,voxelIdx = createVoxelPointsXYDict(np.concatenate((voxels,newVoxels),0))
newFill = fillVoxelMinimalDown(voxelmap, voxelIdx)
newVoxels = np.concatenate((voxels,newFill,newVoxels),0)


doing run 0
[5 5 3]
filling
filling further
translating
(0,)
done
doing run 1
[9 9 5]
filling
filling further
translating
(0,)
done
doing run 2
[19 19 11]
filling
filling further
translating
(33, 3)
done
doing run 3
[37 37 21]
filling
filling further
done


In [10]:
vrgb = voxelGrid2Colors(voxel_grid)

newVoxelCloud = o3d.geometry.PointCloud()
newVoxelCloud.points = o3d.utility.Vector3dVector(np.concatenate((newVoxels,voxels),0))
newVoxelCloud.colors = o3d.utility.Vector3dVector(np.concatenate((np.zeros_like(newVoxels),vrgb),0))

voxel_grid_new = o3d.geometry.VoxelGrid.create_from_point_cloud(newVoxelCloud,
                                                            voxel_size=voxelSize)

o3d.visualization.draw_geometries([voxel_grid_new])

In [11]:
def makeVoxelDict(array):
    vdict = {}
    for i in range(len(array)):
        vdict[(array[i][0],array[i][1],array[i][2])] = {"color":[0,0,0]}
    return vdict
    
voxelFinal =    makeVoxelDict(np.concatenate((newVoxels,voxels),0))

In [12]:
#meshing and creating surfaceVoxels
def vertexKey(n1,n2):
    nbs = [n1,n2]
    return tuple(np.array(nbs).mean(axis=0))
def meshIt(voxelFinal):
    vertices = {}
    vertexList = []
    faces = []
    face_normals = []
    face_colors = []

    surfaceVoxels = {}
    for key in voxelFinal:
        normalDirection = np.array([0,0,0])
        voxelThere = False
        surfaceDirections = []
        if tuple((key[0]-1,key[1],key[2])) not in voxelFinal:
            normalDirection[0] -= 1
            voxelThere = True
            surfaceDirections.append(tuple((key[0]-1,key[1],key[2])))
        if tuple((key[0]+1,key[1],key[2])) not in voxelFinal:
            normalDirection[0] += 1
            voxelThere = True
            surfaceDirections.append(tuple((key[0]+1,key[1],key[2])))
        if tuple((key[0],key[1]-1,key[2])) not in voxelFinal:
            normalDirection[1] -= 1
            voxelThere = True
            surfaceDirections.append(tuple((key[0],key[1]-1,key[2])))
        if tuple((key[0],key[1]+1,key[2])) not in voxelFinal:
            normalDirection[1] += 1
            voxelThere = True
            surfaceDirections.append(tuple((key[0],key[1]+1,key[2])))
        if tuple((key[0],key[1],key[2]-1)) not in voxelFinal:
            normalDirection[2] -= 1
            voxelThere = True
            surfaceDirections.append(tuple((key[0],key[1],key[2]-1)))
        if tuple((key[0],key[1],key[2]+1)) not in voxelFinal:
            normalDirection[2] += 1
            voxelThere = True
            surfaceDirections.append(tuple((key[0],key[1],key[2]+1)))
        if voxelThere:
            if abs(normalDirection).sum() > 0:
                surfaceVoxels[key] = {"normal":normalDirection}
                selfKey = np.array(key)
                for direction in surfaceDirections:
                    #two triangle-faces per direction
                    dirKey = np.array(direction)
                    face_normals.append(dirKey-selfKey)
                    face_normals.append(dirKey-selfKey)
                    face_colors.append(voxelFinal[key]["color"])
                    face_colors.append(voxelFinal[key]["color"])
                    variableList = []
                    fixedIdx = -1
                    fixedDirection = 0
                    for i in range(3):
                        if face_normals[-1][i] == 0:
                            variableList.append(i)
                        else: 
                            fixedIdx = i
                            fixedDirection = face_normals[-1][i]
                    #face1vertices
                    vtx1nbr = np.array(key)
                    vtx1nbr[fixedIdx] += fixedDirection
                    vtx1nbr[variableList[0]] += 1
                    vtx1nbr[variableList[1]] += 1

                    vtx2nbr = np.array(key)
                    vtx2nbr[fixedIdx] += fixedDirection
                    vtx2nbr[variableList[0]] += -1
                    vtx2nbr[variableList[1]] += 1

                    vtx3nbr = np.array(key)
                    vtx3nbr[fixedIdx] += fixedDirection
                    vtx3nbr[variableList[0]] += -1
                    vtx3nbr[variableList[1]] += -1

                    #face2vertices
                    Bvtx1nbr = np.array(key)
                    Bvtx1nbr[fixedIdx] += fixedDirection
                    Bvtx1nbr[variableList[0]] += -1
                    Bvtx1nbr[variableList[1]] += -1

                    Bvtx2nbr = np.array(key)
                    Bvtx2nbr[fixedIdx] += fixedDirection
                    Bvtx2nbr[variableList[0]] += 1
                    Bvtx2nbr[variableList[1]] += -1

                    Bvtx3nbr = np.array(key)
                    Bvtx3nbr[fixedIdx] += fixedDirection
                    Bvtx3nbr[variableList[0]] += 1
                    Bvtx3nbr[variableList[1]] += 1

                    vtxIdx = []

                    #getVertexIdx
                    for vtxNbr in [vtx1nbr,vtx2nbr,vtx3nbr,Bvtx1nbr,Bvtx2nbr,Bvtx3nbr]:
                        vtxKey = vertexKey(list(vtxNbr),list(key))
                        if vtxKey not in vertices:
                            #create vtx
                            vertices[vtxKey] = len(vertexList)
                            vertexList.append(list(vtxKey))
                        vtxIdx.append(vertices[vtxKey])
                    #bugfix if face-normals get ignored
                    if face_normals[-1][0] > 0: #x dir
                        faces.append([vtxIdx[0],vtxIdx[1],vtxIdx[2]])
                        faces.append([vtxIdx[3],vtxIdx[4],vtxIdx[5]])
                    elif face_normals[-1][0] < 0: #x dir
                        faces.append([vtxIdx[2],vtxIdx[1],vtxIdx[0]])
                        faces.append([vtxIdx[5],vtxIdx[4],vtxIdx[3]])
                    elif face_normals[-1][1] < 0: #y dir
                        faces.append([vtxIdx[0],vtxIdx[1],vtxIdx[2]])
                        faces.append([vtxIdx[3],vtxIdx[4],vtxIdx[5]])
                    elif face_normals[-1][1] > 0: #y dir
                        faces.append([vtxIdx[2],vtxIdx[1],vtxIdx[0]])
                        faces.append([vtxIdx[5],vtxIdx[4],vtxIdx[3]])
                    elif face_normals[-1][2] > 0: #z dir
                        faces.append([vtxIdx[0],vtxIdx[1],vtxIdx[2]])
                        faces.append([vtxIdx[3],vtxIdx[4],vtxIdx[5]])
                    elif face_normals[-1][2] < 0: #z dir
                        faces.append([vtxIdx[2],vtxIdx[1],vtxIdx[0]])
                        faces.append([vtxIdx[5],vtxIdx[4],vtxIdx[3]])

            else:
                continue
    voxelPoints = np.array(list(surfaceVoxels.keys()))
    bounds = voxelPoints.max(axis=0)
    voxelNormals = np.zeros_like(voxelPoints)

    voxelPoints = np.array(list(surfaceVoxels.keys()))
    bounds = voxelPoints.max(axis=0)
    voxelNormals = np.zeros_like(voxelPoints)
    for i,key in enumerate(surfaceVoxels.keys()):
        voxelNormals[i] = surfaceVoxels[key]["normal"]
    return vertexList,faces,face_normals,face_colors,surfaceVoxels,bounds

vertexList,faces,face_normals,face_colors,surfaceVoxels,bounds = meshIt(voxelFinal)

In [13]:

meshedMesh = tm.Trimesh(vertices=vertexList, faces=faces, face_normals=None, vertex_normals=None, face_colors=face_colors, vertex_colors=None, face_attributes=None, vertex_attributes=None, metadata=None, process=True, validate=False, use_embree=True, initial_cache=None, visual=None)

In [14]:
meshedMesh.show()

/home/jhm/.local/lib/python3.6/site-packages/IPython/core/display.py:701: UserWarning: Consider using IPython.display.IFrame instead
  warnings.warn("Consider using IPython.display.IFrame instead")


In [15]:
meshedMesh.export("bla.obj")
print("exported")

exported


In [16]:
import subprocess

inputfile = "/home/jhm/Desktop/Arbeit/ConvexNeuralVolume/bla.obj"
outputfile = "/home/jhm/Desktop/Arbeit/ConvexNeuralVolume/bla_vhacd2.obj"
resolution = 200000 #maximum number of voxels generated during the voxelization stage
depth = 32 #maximum number of clipping stages. During each split stage, all the model parts (with a concavity higher than the user defined threshold) are clipped according the "best" clipping plane 	20 	1-32
concavity =0.05 #maximum concavity 	0.0025 	0.0-1.0
planeDownsampling = 2 #controls the granularity of the search for the "best" clipping plane 	4 	1-16
convexhullDownsampling = 1 #controls the precision of the convex-hull generation process during the clipping plane selection stage 	4 	1-16
alpha = 0.05 #controls the bias toward clipping along symmetry planes 	0.05 	0.0-1.0
beta = 0.05 #controls the bias toward clipping along revolution axes 	0.05 	0.0-1.0
gamma = 0.005 #maximum allowed concavity during the merge stage 	0.00125 	0.0-1.0
pca = 0 #enable/disable normalizing the mesh before applying the convex decomposition 	0 	0-1
mode = 0 #voxel-based approximate convex decomposition, 1: tetrahedron-based approximate convex decomposition 	0 	0-1
maxNumVerticesPerCH = 16 #controls the maximum number of triangles per convex-hull 	64 	4-1024
minVolumePerCH = 0.000001 #controls the adaptive sampling of the generated convex-hulls 	0.0001 	0.0-0.01

vhacdPath = "/home/jhm/v-hacd-master/src/build/test/testVHACD"

subprocess.call("{} --input '{}' --output '{}' --resolution {} --depth {} --concavity {} --planeDownsampling {} --convexhullDownsampling {} --alpha {} --beta {} --gamma {} --pca {} --mode {} --maxNumVerticesPerCH {} --minVolumePerCH {}".format(vhacdPath,
                   inputfile,outputfile,resolution,depth,concavity, planeDownsampling,convexhullDownsampling,alpha,beta,gamma,pca,mode,maxNumVerticesPerCH,minVolumePerCH), shell=True, stdout=subprocess.PIPE)

0

In [17]:
mesh2 = tm.load("bla_vhacd2.obj")
meshes = mesh2.split()
mesh2.show()

In [28]:
# capture every object in its H-Representation
import convexPoints2HRepresentation

In [19]:
import torch
points = torch.zeros((len(meshes),16,3))
for i in range(len(meshes)):
    points[i] = torch.tensor(meshes[i].vertices)

In [29]:
planeHrep, _,_ = convexPoints2HRepresentation.step_withoutNaN(convexPoints2HRepresentation.startPlanes, points.cuda())

AttributeError: module 'convexPoints2HRepresentation' has no attribute 'step_withoutNaN'

In [30]:
planeHrep.shape

torch.Size([17, 14, 3])

# create empty points and fuse them in a region block

In [31]:
#is everything thats empty empty? -> no .... but there is no other good way
newVoxels

array([[16., 28., 15.],
       [ 6., 11.,  9.],
       [16., 28., 16.],
       ...,
       [39., 39.,  1.],
       [39., 39.,  2.],
       [39., 39.,  3.]])

In [34]:
vertexListMinBounds = np.array(vertexList).min(axis=0)
vertexListMaxBounds = np.array(vertexList).max(axis=0)

In [55]:
notEmpty = set(voxelFinal.keys())
empty = set()
for x in range(int((vertexListMaxBounds+vertexListMinBounds)[0])):
    for y in range(int((vertexListMaxBounds+vertexListMinBounds)[1])):
        for z in range(int((vertexListMaxBounds+vertexListMinBounds)[2])):
            if (x,y,z) not in notEmpty:
                empty.add((x,y,z))
empty = np.array(list(empty),dtype=np.float)

# move cells to correct position

In [63]:
#bring vertices from grid-coordinates back to pointcloud-coordinates
minBound = np.asarray(pcd.points).min(axis=0)
maxBound = np.asarray(pcd.points).max(axis=0)

mesh2 = tm.load("bla_vhacd2.obj")
zeroed = np.array(mesh2.vertices)+vertexListMinBounds
empty_z = empty+vertexListMinBounds

zeroed = zeroed/(vertexListMaxBounds-vertexListMinBounds)
empty_z = empty_z/(vertexListMaxBounds-vertexListMinBounds)


mesh2.vertices = zeroed*(maxBound-minBound)+minBound
empty_z = empty_z*(maxBound-minBound)+minBound

meshes = mesh2.split()


/home/jhm/.local/lib/python3.6/site-packages/IPython/core/display.py:701: UserWarning: Consider using IPython.display.IFrame instead
  warnings.warn("Consider using IPython.display.IFrame instead")


In [64]:
starts = []
sizes = []
for m in meshes:
    starts.append(m.bounds.mean(axis=0))
    sizes.append((meshes[0].bounds[1]-meshes[0].bounds[0]).max())

# write empty blocks, refined blocks, neural Volume starts

In [65]:
#write empty Blocks (up to 27000 per 30*30*30 volume)
def appendEmptyPoints(x_block,y_block,z_block, points):
    my_file = Path(pfad+"/emptyBlocks/e_{}x{}y{}z.npy".format(x_block,y_block,z_block))
    if my_file.is_file():
        #load existing Points and append new ones
        oldpoints = np.load(my_file)
        np.save(my_file,np.concatenate([oldpoints,points],0))
    else:
        np.save(my_file,points)
        
pointDicts = {}
for p in empty_z:
    block = (int(p[0]//5),int(p[1]//5),int(p[2]//5))
    if block in pointDicts:
        pointDicts[block].append(p)
    else:
        pointDicts[block] = [p]
for key in pointDicts.keys():
    appendEmptyPoints(key[0],key[1],key[2], np.stack(pointDicts[key],0))

In [83]:
#write startVolumes
for i in range(len(planeHrep)):
    my_file = Path(pfad+"/neuralVolumes/{}x{}y{}z.npy".format(int(starts[i][0]//5),
                                                              int(starts[i][1]//5),
                                                              int(starts[i][2]//5)))
    np.save(my_file,np.concatenate([starts[i][None,:],planeHrep[i].cpu()],0))

In [88]:
#save refined Points
refPoints = np.concatenate([np.asarray(cl.points),np.asarray(cl.colors)],1)

pointDicts = {}
for p in refPoints:
    block = (int(p[0]//5),int(p[1]//5),int(p[2]//5))
    if block in pointDicts:
        pointDicts[block].append(p)
    else:
        pointDicts[block] = [p]
for key in pointDicts.keys():
    my_file = Path(pfad+"/blocks/{}x{}y{}z.npy".format(key[0],key[1],key[2]))
    np.save(my_file,np.stack(pointDicts[key],0))